# Data Formatting and Feature Engineering

This notebook will walkthrough the process of cleaning and wrangling our data, doing some feature engineering, and generally getting a good sense of our data.

We first join features from various tables. Various datasets were collated, and as our data contains related tables, it seemed logical to query the data and take only what was needed for each dataset. Having various datasets helped to simplify the EDA process, and to enable easy visualisation building with Tableau.



In [1]:
import pandas as pd
import pickle
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
sellers_df = pd.read_csv('data/source_data/olist_sellers_dataset.csv')
products_df = pd.read_csv('data/source_data/olist_products_dataset.csv')
orders_df = pd.read_csv('data/source_data/olist_orders_dataset.csv')
reviews_df = pd.read_csv('data/source_data/olist_order_reviews_dataset.csv')
payments_df = pd.read_csv('data/source_data/olist_order_payments_dataset.csv')
items_df = pd.read_csv('data/source_data/olist_order_items_dataset.csv')
geo_df = pd.read_csv('data/source_data/olist_geolocation_dataset.csv')
customers_df = pd.read_csv('data/source_data/olist_customers_dataset.csv')
translation_df = pd.read_csv('data/source_data/product_category_name_translation.csv')

#### Products

Joining features relating to the products sold.

In [3]:
q = """SELECT p.product_id, t.product_category_name_english, p.product_name_lenght, p.product_description_lenght, p.product_photos_qty, p.product_weight_g, p.product_length_cm, p.product_height_cm, p.product_width_cm,
               o.order_id, o.customer_id, o.order_purchase_timestamp, o.order_approved_at, o.order_delivered_carrier_date, o.order_delivered_customer_date, o.order_estimated_delivery_date,
               i.order_item_id, i.shipping_limit_date, i.price, i.freight_value,
               s.seller_id, s.seller_zip_code_prefix, s.seller_city, s.seller_state,
               r.review_score
        FROM products_df p
        INNER JOIN items_df i
        ON i.product_id = p.product_id
        INNER JOIN sellers_df s
        ON s.seller_id = i.seller_id
        INNER JOIN orders_df o
        ON o.order_id = i.order_id
        INNER JOIN reviews_df r
        ON o.order_id = r.order_id
        INNER JOIN translation_df t
        ON p.product_category_name = t.product_category_name
        ;"""

df = pysqldf(q)

df.head()

df.drop_duplicates(subset=['customer_id','order_id','product_id'], keep='last', inplace=True)
df.reset_index(inplace=True)

#### Delivery

Columns are converted to DateTime Objects, and features were extracted to give insight on order approval and delivery times.

In [4]:
date_cols = ['order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date','shipping_limit_date']

for col in date_cols:
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True, errors='coerce')

In [5]:
approval_time = []

for x, time in enumerate(df['order_approved_at']):
    approval_time.append(time - df['order_purchase_timestamp'][x])

In [6]:
total_order_time = []

for x, time in enumerate(df['order_delivered_customer_date']):
    total_order_time.append(time - df['order_purchase_timestamp'][x])

In [7]:
order_estimated_delivery = []

for x, time in enumerate(df['order_purchase_timestamp']):
    order_estimated_delivery.append(time - df['order_estimated_delivery_date'][x])

In [8]:
order_estimation_difference = []

for x, time in enumerate(df['order_delivered_customer_date']):
    order_estimation_difference.append(time - df['order_estimated_delivery_date'][x])

In [9]:
order_shipping_limit = []

for x, time in enumerate(df['shipping_limit_date']):
    order_shipping_limit.append(time - df['order_purchase_timestamp'][x])

In [10]:
df['order_approval_time'] = approval_time
df['total_order_time'] = total_order_time
df['order_estimated_delivery'] = order_estimated_delivery
df['order_estimation_difference'] = order_estimation_difference
df['order_shipping_limit'] = order_shipping_limit

In [11]:
df.drop(columns=date_cols, inplace=True)

#### Delivery Averages per Product

Averages were collected and joined with original DataFrame.

In [12]:
delivery_df = df[['product_id', 'product_category_name_english', 'order_approval_time', 'total_order_time', 'order_estimated_delivery',
                  'order_estimation_difference', 'order_shipping_limit','order_id']]

delivery_averages_ = delivery_df.drop(columns=['order_id']).groupby('product_id').agg(lambda x: x.mean())
delivery_averages = delivery_averages_.copy()

In [13]:
data = df.join(delivery_averages, on='product_id', rsuffix='_average')

data[['customer_id','product_id','order_approval_time','order_approval_time_average',
    'total_order_time','total_order_time_average','order_estimated_delivery',
    'order_estimated_delivery_average','order_estimation_difference','order_estimation_difference_average','order_shipping_limit',
    'order_shipping_limit_average']].head(10)

,customer_id,product_id,order_approval_time,order_approval_time_average,total_order_time,total_order_time_average,order_estimated_delivery,order_estimated_delivery_average,order_estimation_difference,order_estimation_difference_average,order_shipping_limit,order_shipping_limit_average
0,f8a3e963a310aa58b60a5b1fed5bceb5,1e9e8ef04dbcff4541ed26657ea517e5,0 days 02:47:26,0 days 02:47:26,3 days 00:25:24,3 days 00:25:24,-15 days +16:16:53,-15 days +16:16:53,-12 days +16:42:17,-12 days +16:42:17,6 days 01:17:01,6 days 01:17:01
1,03fc97548af8f58fefc768d12b546c9c,3aa071139cb16b67ca9e5dea641aaa2f,0 days 00:15:39,0 days 00:15:39,15 days 18:31:32,15 days 18:31:32,-22 days +18:55:36,-22 days +18:55:36,-6 days +13:27:08,-6 days +13:27:08,6 days 00:15:39,6 days 00:15:39
2,e41819d1c95c12c9ce495b630eab8aee,96bd76ec8810374ed1b65e291975717f,0 days 00:12:19,0 days 00:12:19,2 days 02:06:15,2 days 02:06:15,-9 days +21:18:01,-9 days +21:18:01,-7 days +23:24:16,-7 days +23:24:16,2 days 00:12:19,2 days 00:12:19
3,322162b5ca010c2b059cb5224dd818b1,cef67bcfe19066a932b7673e239eb23d,0 days 00:35:52,0 days 00:35:52,3 days 06:18:42,3 days 06:18:42,-5 days +08:55:50,-5 days +08:55:50,-2 days +15:14:32,-2 days +15:14:32,4 days 00:14:23,4 days 00:14:23
4,c11c31965ff02cc1d7132df8edfcbc22,9dc1a7de274444849c219cff195d0b71,0 days 00:23:46,0 days 00:23:46,2 days 21:35:07,2 days 21:35:07,-15 days +01:06:37,-15 days +01:06:37,-13 days +22:41:44,-13 days +22:41:44,6 days 00:23:46,6 days 00:23:46
5,3f3a10ec410c8efb0ca799fa7705d07e,41d3672d4792049fa1779bb35283ed13,1 days 00:06:48,1 days 00:06:48,17 days 02:58:00,17 days 02:58:00,-36 days +15:41:41,-36 days +15:41:41,-19 days +18:39:41,-19 days +18:39:41,7 days 00:06:48,7 days 00:06:48
6,7e395b4bf53aea8c56b1213f1b801411,732bd381ad09e530fe0a5f457d81becb,0 days 00:08:09,0 days 00:27:27.500000,5 days 10:07:46,9 days 08:56:01,-24 days +12:31:15,-27 days +23:53:06.500000,-19 days +22:39:01,-17 days +08:49:07.500000,4 days 00:08:09,5 days 00:12:01
7,30f840412466fa26e5c035e1ad1fdec3,732bd381ad09e530fe0a5f457d81becb,0 days 00:46:46,0 days 00:27:27.500000,13 days 07:44:16,9 days 08:56:01,-29 days +11:14:58,-27 days +23:53:06.500000,-16 days +18:59:14,-17 days +08:49:07.500000,6 days 00:15:53,5 days 00:12:01
8,181e2217f874cb5390be37f378db5e07,2548af3e6e77a690cf3eb6368e9ab61e,0 days 02:10:55,0 days 00:50:43.333333,12 days 19:15:42,8 days 18:43:02.333333,-25 days +14:22:48,-22 days +05:30:05,-12 days +09:38:30,-13 days +00:13:07.333333,6 days 02:08:12,4 days 16:49:49
9,2ea5983ec2edde5dcba726ea64e9fadb,2548af3e6e77a690cf3eb6368e9ab61e,0 days 00:09:27,0 days 00:50:43.333333,8 days 10:05:55,8 days 18:43:02.333333,-23 days +11:26:14,-22 days +05:30:05,-15 days +21:32:09,-13 days +00:13:07.333333,4 days 00:09:27,4 days 16:49:49


In [14]:
products = data
# products.to_csv('data/products.csv', index=False)

In [15]:
pickle.dump(products, open('data/products.p', 'wb'))

# Customers

Combined with information about the product(s) bought, this data can be used for Clustering for better insight on customers.

Freight Value %?

In [16]:
q = """SELECT c.customer_id, p.order_id, c.customer_zip_code_prefix, c.customer_city, 
                c.customer_state,
                p.payment_value, i.price, i.freight_value, i.product_id,
                p.payment_type, p.payment_installments, i.order_item_id, 
                t.product_category_name_english,
                s.seller_city, s.seller_state
        FROM customers_df c
        INNER JOIN orders_df o
        ON o.customer_id = c.customer_id
        INNER JOIN payments_df p
        ON o.order_id = p.order_id
        INNER JOIN items_df i
        ON o.order_id = i.order_id
        INNER JOIN products_df pd
        ON i.product_id = pd.product_id
        INNER JOIN sellers_df s
        ON i.seller_id = s.seller_id
        INNER JOIN translation_df t
        ON pd.product_category_name = t.product_category_name
        ;"""

data = pysqldf(q)

data['items_in_order'] = data['order_item_id']
data.drop(columns=['order_item_id'], inplace=True)

data.drop_duplicates(subset=['customer_id','order_id','product_id'], keep='last', inplace=True)
data.reset_index(inplace=True)

In [17]:
data['freight_total_pay'] = data['freight_value'] / data['payment_value'] 

In [18]:
data['freight_item_price'] = data['freight_value'] / data['price']

In [19]:
customers = data
# customers.to_csv('data/customers.csv', index=False)
pickle.dump(customers, open('data/customers.p', 'wb'))

# Reviews

Potential for Sentiment Analysis

Length of review, creation date

In [20]:
q = """SELECT r.review_id, r.order_id, r.review_score, r.review_comment_title, r.review_comment_message
        FROM reviews_df r
        ;"""

data = pysqldf(q)

In [21]:
title_lengths = []

for x, length in enumerate(data['review_comment_title']):
    if length == None:
        title_lengths.append('None')
    else:
        title_lengths.append(len(length))
        
        
review_lengths = []

for x, length in enumerate(data['review_comment_message']):
    if length == None:
        review_lengths.append('None')
    else:
        review_lengths.append(len(length))
    
data['title_length'] = title_lengths
data['review_length'] = review_lengths

In [22]:
reviews = data
# reviews.to_csv('data/reviews.csv', index=False)
pickle.dump(reviews, open('data/reviews.p', 'wb'))

# Data Prep

## Full Dataset

A dataset was built to include all the relevent features or our regression model.

In [23]:
products = pd.read_csv('data/products.csv')
customers = pd.read_csv('data/customers.csv')
reviews = pd.read_csv('data/reviews.csv')

In [24]:
customers['customer_id'] = customers['customer_id'].astype('category')
products[['customer_id', 'order_id']] = products[['customer_id','order_id']].astype('category')
reviews['order_id'] = reviews['order_id'].astype('category')

In [25]:
customers.drop_duplicates(subset=['customer_id','order_id','product_id'], keep='last', inplace=True)
customers.reset_index(inplace=True)

products.drop_duplicates(subset=['customer_id','order_id','product_id'], keep='last', inplace=True)
products.reset_index(inplace=True)

In [26]:
indx = []

for x in customers.index:
    indx.append(','.join([customers['customer_id'][x], customers['order_id'][x]]))
    
    
customers['indx'] = indx

indxx = []

for x in products.index:
    indxx.append(','.join([products['customer_id'][x], products['order_id'][x]]))
    
products['indx'] = indxx

customers.set_index('indx', inplace=True)
products.set_index('indx', inplace=True)

In [27]:
dff = customers.join(products, rsuffix='_dupe')
df = dff.join(reviews, rsuffix='_dupe')

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
cols = list(df.columns)
string = '_dupe'

for x in cols:
    if string in x:
        df.drop(columns=[x], inplace=True)
    else:
        continue

## Data Types



Revew columns dropped - too many Nans

In [ ]:
df.drop(columns=[ 'review_id','review_comment_title','review_comment_message','title_length','review_length','order_item_id'], inplace=True)

In [ ]:
df.drop(columns=['level_0','index','customer_id','order_id','product_id','indx','seller_id'], inplace=True)

In [ ]:
df.drop(columns=['seller_zip_code_prefix','customer_zip_code_prefix'],inplace=True)

In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [ ]:
cols = ['payment_type','customer_state','product_category_name_english','seller_state','review_score']

for col in cols:
    df[col] = df[col].astype('category')

In [ ]:
# df.to_csv('data/full_dataset.csv')
pickle.dump(df, open('data/full_dataset.p', 'wb'))

In [ ]:
df_dum = df[['product_category_name_english','customer_state','seller_state','review_score','payment_type']]

dff = pd.get_dummies(df_dum)

In [ ]:
final = pd.concat([df, dff], axis=1)
final.drop(columns=['product_category_name_english','customer_state','seller_state','review_score','payment_type'],
          inplace=True)

In [ ]:
pickle.dump(final, open('data/final_df_dummies.p', 'wb'))